In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from tqdm import tqdm
import sys
import os
sys.path.append(os.path.abspath('../common/'))
import common_functions as cf
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, train_test_split
from sklearn.linear_model import LinearRegression, Ridge, ElasticNet, BayesianRidge, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.isotonic import IsotonicRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import RepeatedKFold, GridSearchCV
from sklearn.svm import SVR

In [2]:
weather_path = 'Input/weather_train_set0.csv'
demand_path = 'Input/demand_train_set0.csv'
solar_path = 'Input/pv_train_set0.csv'
dp = cf.DataPreprocesser(weather_path, demand_path, solar_path)
dp.load_df()
dp.remove_nan()
dp.interpolate_df()
dp.set_df(dp.df[dp.df['week']!=44])
dp.df.head()

,demand_MW,irradiance_Wm-2,pv_power_mw,panel_temp_C,temp_location3,temp_location6,temp_location2,temp_location4,temp_location5,temp_location1,solar_location3,solar_location6,solar_location2,solar_location4,solar_location5,solar_location1,week,dow,hour,sp
datetime,,,,,,,,,,,,,,,,,,,,
2017-11-06 00:00:00,2.32,0.0,0.0,1.61,5.14,10.76,6.350,3.58,11.120,7.390,0.0,0.0,0.0,0.0,0.0,0.0,45,0,0,1.0
2017-11-06 00:30:00,2.22,0.0,0.0,2.98,5.05,10.70,6.245,3.58,11.060,7.325,0.0,0.0,0.0,0.0,0.0,0.0,45,0,0,2.0
2017-11-06 01:00:00,2.04,0.0,0.0,2.76,4.96,10.64,6.140,3.58,11.000,7.260,0.0,0.0,0.0,0.0,0.0,0.0,45,0,1,3.0
2017-11-06 01:30:00,1.97,0.0,0.0,3.98,5.03,10.60,6.085,3.57,10.965,7.255,0.0,0.0,0.0,0.0,0.0,0.0,45,0,1,4.0
2017-11-06 02:00:00,1.91,0.0,0.0,2.13,5.10,10.56,6.030,3.56,10.930,7.250,0.0,0.0,0.0,0.0,0.0,0.0,45,0,2,5.0


In [5]:
dp.df['week'].unique().size

37

In [102]:
from sklearn.model_selection import TimeSeriesSplit
week_list = dp.df['week'].unique()
week_nb = week_list.size
tscv = TimeSeriesSplit(n_splits=5)
splits = pd.DataFrame(columns=['week_train', 'week_test'])
idx = 0
for ind_train, ind_test in tscv.split(np.arange(week_nb)):
    splits.loc[idx, :] = [week_list[ind_train], week_list[ind_test]]
    idx += 1
splits

/home/raph/anaconda3/envs/westernpower/lib/python3.7/site-packages/pandas/core/internals/blocks.py:983: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  arr_value = np.array(value)


,week_train,week_test
0,"[45, 46, 47, 48, 49, 50, 51]","[52, 1, 2, 3, 4, 5]"
1,"[45, 46, 47, 48, 49, 50, 51, 52, 1, 2, 3, 4, 5]","[6, 7, 8, 9, 10, 11]"
2,"[45, 46, 47, 48, 49, 50, 51, 52, 1, 2, 3, 4, 5...","[12, 13, 14, 15, 16, 17]"
3,"[45, 46, 47, 48, 49, 50, 51, 52, 1, 2, 3, 4, 5...","[18, 19, 20, 21, 22, 23]"
4,"[45, 46, 47, 48, 49, 50, 51, 52, 1, 2, 3, 4, 5...","[24, 25, 26, 27, 28, 29]"


In [4]:
from sklearn.metrics import mean_squared_error
import xgboost as xgb

In [44]:
xgbtuned = xgb.XGBRegressor(n_estimators =50, gamma=0.25, learning_rate=0.1, max_depth=10, colsample_bylevel=0.5, min_child_weight=3.0)
param_grid={
#         'max_depth': [3,  5, 6, 10,20,50]
#         'learning_rate': [0.01, 0.1, 0.2, 0.3]
#         'subsample': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
#         'colsample_bytree': [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
#         'colsample_bylevel': [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
        'min_child_weight': [0.5, 1.0, 3.0, 5.0, 7.0, 10.0]
#         'gamma': [0, 0.25, 0.5, 1.0]
#         'n_estimators': [10, 50, 100, 200, 300, 400]
}
scores = []
# for key in param_grid.keys():
#     for param in param_grid[key]:
#         for ind in 


In [27]:
len(param_grid['gamma'])

4

In [35]:
splits.shape[0]

5

In [45]:
weather_columns = dp.get_columns_of_group_names(['temp'], [1,2])
weather_columns.append('sp')
weather_columns.append('dow')
X,y = dp.build_input_for_ml_algo(weather_columns, ['demand_MW'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=2019)

In [46]:
search = GridSearchCV(xgbtuned, param_grid, cv=5)
search.fit(X_train, y_train)
search.score

<bound method BaseSearchCV.score of GridSearchCV(cv=5,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    colsample_bylevel=0.5,
                                    colsample_bynode=None,
                                    colsample_bytree=None, gamma=0.25,
                                    gpu_id=None, importance_type='gain',
                                    interaction_constraints=None,
                                    learning_rate=0.1, max_delta_step=None,
                                    max_depth=10, min_child_weight=None,
                                    missing=nan, monotone_constraints=None,
                                    n_estimators=50, n_jobs=None,
                                    num_parallel_tree=None, random_state=None,
                                    reg_alpha=None, reg_lambda=None,
                                    scale_pos_weight=None, subsample=None,
                                    

In [47]:
print(search.best_score_)
search.best_params_

0.8290884862226757


{'min_child_weight': 3.0}

In [50]:
prediction_week = 30

In [49]:
ml_predictor = cf.MLPredictor(dp,prediction_week)
ml_predictor.get_demand_previous_week()
ml_predictor.get_weather_prediction(weather_path)
# ml_predictor.predict_solar_power_from_weather(xgb.XGBRegressor(n_estimators =50, gamma=0.25, learning_rate=0.1, 
#                                                                max_depth=10, colsample_bylevel=0.5, min_child_weight=3.0))

/home/raph/project/westernpower/westernpower/task0/common_functions.py:296: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series.  To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  demand_prediction['week']=demand_prediction.index.week


,demand_MW,week,dow,hour,sp,temp_location3,temp_location6,temp_location2,temp_location4,temp_location5,temp_location1,solar_location3,solar_location6,solar_location2,solar_location4,solar_location5,solar_location1
datetime,,,,,,,,,,,,,,,,,
2018-07-23 00:00:00,1.73,30.0,0.0,0.0,1.0,15.670,18.500,17.050,17.860,18.740,16.740,0.0,0.0,0.0,0.0,0.0,0.0
2018-07-23 00:30:00,1.62,30.0,0.0,0.0,2.0,15.570,18.355,16.845,17.585,18.630,16.620,0.0,0.0,0.0,0.0,0.0,0.0
2018-07-23 01:00:00,1.56,30.0,0.0,1.0,3.0,15.470,18.210,16.640,17.310,18.520,16.500,0.0,0.0,0.0,0.0,0.0,0.0
2018-07-23 01:30:00,1.48,30.0,0.0,1.0,4.0,15.370,18.110,16.485,17.050,18.455,16.410,0.0,0.0,0.0,0.0,0.0,0.0
2018-07-23 02:00:00,1.48,30.0,0.0,2.0,5.0,15.270,18.010,16.330,16.790,18.390,16.320,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-07-29 21:30:00,2.40,30.0,6.0,21.0,44.0,15.485,17.415,16.370,16.285,17.625,16.125,0.0,0.0,0.0,0.0,0.0,0.0
2018-07-29 22:00:00,2.15,30.0,6.0,22.0,45.0,15.310,17.390,16.300,16.030,17.580,16.040,0.0,0.0,0.0,0.0,0.0,0.0
2018-07-29 22:30:00,1.93,30.0,6.0,22.0,46.0,15.215,17.380,16.255,15.835,17.570,15.980,0.0,0.0,0.0,0.0,0.0,0.0


In [51]:
predicted_solar_power = dp.df.loc[dp.df['week'] == (prediction_week-1), ['pv_power_mw']]
predicted_solar_power.index = predicted_solar_power.index + pd.DateOffset(7)
predicted_solar_power

,pv_power_mw
datetime,
2018-07-23 00:00:00,0.0
2018-07-23 00:30:00,0.0
2018-07-23 01:00:00,0.0
2018-07-23 01:30:00,0.0
2018-07-23 02:00:00,0.0
...,...
2018-07-29 21:30:00,0.0
2018-07-29 22:00:00,0.0
2018-07-29 22:30:00,0.0


In [52]:
weather_columns = dp.get_columns_of_group_names(['temp'], [1,2,5,6])
weather_columns.append('sp')

In [53]:
X_train = dp.df.loc[(dp.df['week'] >= 25)&(dp.df['week'] <= 29),weather_columns].values
y_train = dp.df.loc[(dp.df['week'] >= 25)&(dp.df['week'] <= 29),'demand_MW'].values

In [54]:
predicted_df = ml_predictor.predicted_df
predicted_df['pv_power_mw'] = predicted_solar_power

In [56]:
model = xgb.XGBRegressor(n_estimators =50, gamma=0.25, learning_rate=0.1, 
                                                                max_depth=10, colsample_bylevel=0.5, min_child_weight=3.0)
model.fit(X_train, y_train)
predicted_df['demand_MW'] = model.predict(ml_predictor.predicted_df[weather_columns].values)

In [57]:
bpd = cf.BatteryPowerDispatcher
B= bpd.get_all_dispatch_in_a_week(bpd,predicted_df, prediction_week)
B

,3000,3001,3002,3003,3004,3005,3006
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
10,0.003687,0.012497,0.004397,0.006050,0.009041,0.000000,0.006098


In [58]:
B.sum()

3000    0.081473
3001    0.040403
3002    0.062361
3003    0.024860
3004    0.100766
3005    0.004880
3006    0.040614
dtype: float64

In [59]:
B_final = bpd.format_dispatching_for_competition(B, predicted_df.index)
B_final.to_csv('Output/test_raph/pred_demand_xgb.csv')

In [64]:
weather_columns = dp.get_columns_of_group_names(['temp', 'solar'], [1,2])
weather_columns.append('sp')
dp.set_df(dp.df[dp.df['pv_power_mw'] > 0])
X,y = dp.build_input_for_ml_algo(weather_columns, ['pv_power_mw'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=2019)


In [100]:
xgbtuned = xgb.XGBRegressor(n_estimators=50, learning_rate = 0.15,max_depth=6)
param_grid={
#         'max_depth': [3,  5, 6, 10,20,50]
#         'learning_rate': [0.01,0.075, 0.1,0.149,0.15,0.151 ,0.2, 0.3]
#         'subsample': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
#         'colsample_bytree': [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
#         'colsample_bylevel': [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
#         'min_child_weight': [0.5, 1.0, 3.0, 5.0, 7.0, 10.0]
        'gamma': [0, 0.05,0.15,0.25, 0.5, 1.0]
#         'n_estimators': [10, 50, 100, 200, 300, 400]
}
scores = []
# for key in param_grid.keys():
#     for param in param_grid[key]:
#         for ind in 


In [101]:
search = GridSearchCV(xgbtuned, param_grid, cv=5)
search.fit(X_train, y_train)
print(search.best_score_)
search.best_params_

0.7825994694930822


{'gamma': 0}

In [103]:
weather_path = 'Input/weather_train_set0.csv'
demand_path = 'Input/demand_train_set0.csv'
solar_path = 'Input/pv_train_set0.csv'
dp = cf.DataPreprocesser(weather_path, demand_path, solar_path)
dp.load_df()
dp.remove_nan()
dp.interpolate_df()
dp.set_df(dp.df[dp.df['week']!=44])
dp.df.head()

/home/raph/project/westernpower/westernpower/task0/common_functions.py:39: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series.  To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  df['week']=df.index.week


,demand_MW,irradiance_Wm-2,pv_power_mw,panel_temp_C,temp_location3,temp_location6,temp_location2,temp_location4,temp_location5,temp_location1,solar_location3,solar_location6,solar_location2,solar_location4,solar_location5,solar_location1,week,dow,hour,sp
datetime,,,,,,,,,,,,,,,,,,,,
2017-11-06 00:00:00,2.32,0.0,0.0,1.61,5.14,10.76,6.350,3.58,11.120,7.390,0.0,0.0,0.0,0.0,0.0,0.0,45,0,0,1.0
2017-11-06 00:30:00,2.22,0.0,0.0,2.98,5.05,10.70,6.245,3.58,11.060,7.325,0.0,0.0,0.0,0.0,0.0,0.0,45,0,0,2.0
2017-11-06 01:00:00,2.04,0.0,0.0,2.76,4.96,10.64,6.140,3.58,11.000,7.260,0.0,0.0,0.0,0.0,0.0,0.0,45,0,1,3.0
2017-11-06 01:30:00,1.97,0.0,0.0,3.98,5.03,10.60,6.085,3.57,10.965,7.255,0.0,0.0,0.0,0.0,0.0,0.0,45,0,1,4.0
2017-11-06 02:00:00,1.91,0.0,0.0,2.13,5.10,10.56,6.030,3.56,10.930,7.250,0.0,0.0,0.0,0.0,0.0,0.0,45,0,2,5.0


In [107]:
weather_columns = dp.get_columns_of_group_names(['temp'], [1,2])
weather_columns.append('sp')
weather_columns.append('dow')
X,y = dp.build_input_for_ml_algo(weather_columns, ['demand_MW'])
param_grid={'n_estimators': np.arange(50, 500, 50).tolist()}
search = GridSearchCV(RandomForestRegressor(), param_grid, cv=5)
search.fit(X_train, y_train)
print(search.best_score_)
search.best_params_

0.7807221939310999


{'n_estimators': 150}